# 4.1 Classifying movie reviews: A binary classification example
1. [IMDB Data](#data)
1. [Preparing Data](#preparing)
1. [Building Model](#building)

<a name="data"></a>
# 4.1.1 The IMDB dataset

In [1]:
from tensorflow.keras.datasets import imdb

(train_data, train_labels), (test_data, test_labels) = imdb.load_data(
    num_words=10000)

17473536/17464789 [==============================] - 4s 0us/step


The argument `num_words=10000` means you’ll only __keep the top 10,000 most frequently occurring words in the training data__. 

Rare words will be discarded. This allows us to work with vector data of manageable size. If we didn’t set this limit, we’d be working with __88,585 unique words in the training data__, which is unnecessarily large. Many of these words only occur in a single sample, and thus can’t be meaningfully used for classification.

The variables `train_data` and `test_data` are lists of reviews; each review is a list of word indices (encoding a sequence of words). `train_labels` and `test_labels` are lists of 0s and 1s, where 0 stands for negative and 1 stands for positive.

In [2]:
print(train_data[0])
print(train_labels[0])

[1, 14, 22, 16, 43, 530, 973, 1622, 1385, 65, 458, 4468, 66, 3941, 4, 173, 36, 256, 5, 25, 100, 43, 838, 112, 50, 670, 2, 9, 35, 480, 284, 5, 150, 4, 172, 112, 167, 2, 336, 385, 39, 4, 172, 4536, 1111, 17, 546, 38, 13, 447, 4, 192, 50, 16, 6, 147, 2025, 19, 14, 22, 4, 1920, 4613, 469, 4, 22, 71, 87, 12, 16, 43, 530, 38, 76, 15, 13, 1247, 4, 22, 17, 515, 17, 12, 16, 626, 18, 2, 5, 62, 386, 12, 8, 316, 8, 106, 5, 4, 2223, 5244, 16, 480, 66, 3785, 33, 4, 130, 12, 16, 38, 619, 5, 25, 124, 51, 36, 135, 48, 25, 1415, 33, 6, 22, 12, 215, 28, 77, 52, 5, 14, 407, 16, 82, 2, 8, 4, 107, 117, 5952, 15, 256, 4, 2, 7, 3766, 5, 723, 36, 71, 43, 530, 476, 26, 400, 317, 46, 7, 4, 2, 1029, 13, 104, 88, 4, 381, 15, 297, 98, 32, 2071, 56, 26, 141, 6, 194, 7486, 18, 4, 226, 22, 21, 134, 476, 26, 480, 5, 144, 30, 5535, 18, 51, 36, 28, 224, 92, 25, 104, 4, 226, 65, 16, 38, 1334, 88, 12, 16, 283, 5, 16, 4472, 113, 103, 32, 15, 16, 5345, 19, 178, 32]
1


Because we’re restricting ourselves to the top 10,000 most frequent words, no word index will exceed 10,000.

In [3]:
max([max(sequence) for sequence in train_data])

9999

For kicks, here’s how you can quickly __decode one of these reviews back to English words__.

In [6]:
# dict mapping words to an int index
word_index = imdb.get_word_index()
reverse_word_index = dict(
    # reverses it, mapping int indices to words
    [(value, key) for (key, value) in word_index.items()])
decoded_review = " ".join(
    # decodes review
    [reverse_word_index.get(i - 3, "?") for i in train_data[0]])

<a name="preparing"></a>
# 4.1.2 Preparing the data

You can’t directly feed lists of integers into a NN. They all have different lengths, but __a NN expects to process contiguous batches of data__. You have to __turn your lists into tensors__. There are 2 ways to do that:

1. __Pad your lists__ so that they all have the same length, turn them into an integer tensor of shape `(samples, max_length)`, and start your model with a layer capable of handling such integer tensors (the `Embedding` layer, which we’ll cover in detail later in the book).

2. __Multi-hot encode your lists__ to turn them into vectors of 0s and 1s. Then you could use a `Dense` layer, capable of handling floating-point vector data, as the first layer in your model.

Let’s go with the latter solution to vectorize the data, which you’ll do manually for maximum clarity.

In [ ]:
import numpy as np

def vectorize_sequences(sequences, dimension=10_000):
    # create all-zero matrix of shape (len(sequences), dimension)
    results = np.zeros((len(sequences), dimension))
    for i, sequence in enumerate(sequences):
        for j in sequences:
            # set specific indices of results[i] to 1s
            results[i, j] = 1.
    return results

x_train = vectorize_sequences(train_data)
x_test = vectorize_sequences(test_data)

Here’s what the samples look like now.

In [ ]:
x_train[0]

You should also __vectorize your labels__.

In [ ]:
y_train = np.asarray(train_labels).astype("float32")
y_test = np.asarray(test_labels).astype("float32")

Now the data is __ready to be fed into a NN__.

<a name="building"></a>
# 4.1.3 Building your model